In [ ]:
import numpy as np
import scipy as sci

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import pandas as pd

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge

In [ ]:
from sklearn.preprocessing import SplineTransformer

# Homework 4

## Generate some Toy Data

In [ ]:
def give_me_nonlinear_data(n=None, x=None, noise=0.25, seed=1):
    np.random.seed(seed)
    if x is None:
        x = np.random.uniform(low=0, high=3, size=n)
        x[0],x[-1] = 0, 3
        y = 3 + (np.sin(3 * x) + 2 * np.sin(2 * x**2) + np.sin(x**3)) / np.exp(x) + np.random.standard_normal(n)*noise       
        return x, y
    else:
        return 3 + (np.sin(3 * x) + 2 * np.sin(2 * x**2) + np.sin(x**3)) / np.exp(x) + np.random.standard_normal(n)*noise     

### Generate Train, Validation and Test data

In [ ]:
x, y = give_me_nonlinear_data(n=300, seed=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.6, random_state=1) 

In [ ]:
xx = np.arange(0,3.1,0.01)
plt.figure(figsize=(12,6))
plt.scatter(x_train, y_train, label='data points')
plt.plot(xx, give_me_nonlinear_data(x=xx, noise=0), color='black', linewidth=3, linestyle='--', label=r'ground truth model $f(x)$')
plt.ylabel('y', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.legend(fontsize=18)
plt.savefig('model.pdf')

# Ridge Regression using Sklearn

Our aim to to fit a curve to the above data and balance between the bias and variance to obtain a good model that generalizes to new (unseen) data points.

First, we will construct a b-spline basis matrix.


In [ ]:
spline = SplineTransformer(degree=3, n_knots=40, include_bias=False).fit(x_train.reshape(-1, 1), y_train)
Xtrain = spline.fit_transform(x_train.reshape(-1, 1))
Xval = spline.fit_transform(x_val.reshape(-1, 1))
Xtest = spline.fit_transform(x_test.reshape(-1, 1))

In [ ]:
Xtrain.shape

Now, we fit a model with a small amount of ridge to predict new data points and plot the corresponding curve.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
clf = Ridge(alpha=0.005)
clf.fit(Xtrain, y_train)

In [ ]:
xx = np.arange(0, 3.0,0.001)
XX = spline.fit_transform(xx.reshape(-1, 1))
yypred = clf.predict(XX)

In [ ]:
yypred.shape

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x_train, y_train, label='data points')
plt.plot(xx, give_me_nonlinear_data(x=xx, noise=0), color='black', linewidth=3, linestyle='--', label=r'ground truth model $f(x)$')
plt.plot(xx, yypred, c='red', linewidth=2, label='regularized curve (not tuned)')
plt.ylabel('y', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.legend(fontsize=18)
plt.savefig('model.pdf')

## Exercise 1: Fit a new regularized model (2 points)

Let's try to find a smoother fit by regularizing the model. 

First, to find a good lambda, we can cross-validate the model's performance on the validation set.

In [ ]:
mse_val = []
alphas = []
for alpha in np.arange(0, 0.7, 0.01):
    reg = # your code
    mse_val.append(np.mean((y_val-reg.predict(Xval))**2))
    alphas.append(alpha)
    
idx = np.argmin(mse_val)
print("alpha=%s, mse val=%s" %(alphas[idx], mse_val[idx]))

In [ ]:
plt.plot(alphas[1:], mse_val[1:])
plt.ylabel('Validation Error')
plt.xlabel('alpha')

And then we use the suggested alpha to fit a new model.

In [ ]:
clf2 = # your code
clf2.fit(Xtrain, y_train)
yypred_reg = clf2.predict(XX)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x_train, y_train, label='data points')
plt.plot(xx, give_me_nonlinear_data(x=xx, noise=0), color='black', linewidth=3, linestyle='--', label=r'ground truth model $f(x)$')
plt.plot(xx, yypred, c='red', linewidth=2, label='fregularized curve (not tuned)')
plt.plot(xx, yypred_reg, c='green', linewidth=3, label='regularized curve (tuned)')

plt.ylabel('y', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.legend(fontsize=18)
plt.savefig('model.pdf')

Finally, let us compare the test error for the unregularized and regularized model.

In [ ]:
print('Unregularized model', np.sum((y_test-clf.predict(Xtest))**2))

In [ ]:
print('Regularized model', np.sum((y_test-clf2.predict(Xtest))**2))

## Exercise 2:  Implement the Ridge Estimator using the SVD 6 points)
Use the Singular Value Decomposition (SVD) of X, to compute the Ridge coefficients. To do so, you can use `numpy.linalg.svd`. 

You can build on top of the ols class that we have developed previously:
* Provide an option to pass the argument `alpha`.
* Compute the SVD and use the formula in the notes to compute the coefficients
* To obtain the same performance as sklearn, you need to rescale x and y!
* Be smart, write a method called `path` that reuses the SVD. The `path` method takes as argument a new alpha, and returns the coefficient vector $\beta$. This method should reuse the SVD, which was computed during calling the `fit` method.

In [ ]:
class myRidge:
    
    def __init__(self, alpha):
        # your code

    def fit(self, x, y):
       # your code    
        
        
    def predict(self, x):
        # your code
    
    def path(self, alpha):
        # your code

Use your algorithm to fit the data.

In [ ]:
reg = myRidge(alpha=0.14)
reg.fit(Xtrain, y_train)

In [ ]:
print('Regularized model', np.sum((y_test-reg.predict(Xtest))**2))

In [ ]:
yypred_myreg = reg.predict(XX)

plt.figure(figsize=(12,6))
plt.scatter(x_train, y_train, label='data points')
plt.plot(xx, give_me_nonlinear_data(x=xx, noise=0), color='black', linewidth=3, linestyle='--', label=r'ground truth model $f(x)$')
plt.plot(xx, yypred_reg, c='red', linewidth=2, label='sklearn')
plt.plot(xx, yypred_myreg, c='green', linewidth=3, label='your implementation')

plt.ylabel('y', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.legend(fontsize=18)
plt.savefig('model.pdf')

Now, compute the ridge path.

In [ ]:
beta = []
for alpha in np.arange(0.1, 5, 0.1):
    coef_ = reg.path(alpha=alpha)
    beta.append(coef_)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.arange(0.1, 5, 0.1), np.asarray(beta), 'o--')
plt.ylim(-2,2)
plt.ylabel('beta')
plt.xlabel('ridge')
plt.axhline(y=0, c='black', linewidth=3, linestyle='dashed')

The ridge path should look similar to what you get when you use sklearn, but the colors of the lines might differ.

In [ ]:
beta = []
for alpha in np.arange(0.1, 5, 0.1):
    skreg = Ridge(alpha=alpha)
    skreg = skreg.fit(Xtrain, y_train)
    beta.append(skreg.coef_)

    
plt.figure(figsize=(12,8))
plt.plot(np.arange(0.1, 5, 0.1), np.asarray(beta), 'o--')
plt.ylim(-2,2)
plt.ylabel('beta')
plt.xlabel('ridge')
plt.axhline(y=0, c='black', linewidth=3, linestyle='dashed')

If you implemented your algorithm correctly, then you should see that your algorithm is much faster then using sklearn!! (Roughly by a factor of about 15!)

In [ ]:
%%timeit
beta = []
for alpha in np.arange(0.1, 5, 0.1):
    coef_ = reg.path(alpha=alpha)
    beta.append(coef_)

In [ ]:
%%timeit
beta = []
for alpha in np.arange(0.1, 5, 0.1):
    skreg = Ridge(alpha=alpha)
    skreg = skreg.fit(Xtrain, y_train)
    beta.append(skreg.coef_)

# Linear Regression on Boston Housing Dataset

* First read the following blog post: https://towardsdatascience.com/linear-regression-on-boston-housing-dataset-f409b7e4a155

* Do an Exploratory Data Analysis similar to what is described in the blog post and briefly summarize your observations.

* Then, build a simple model that uses only 3 variables and interpret the results. To get full points, your model should have an MSE that is equal to or lower than 4100.

* Finally, build a predictive model that best predicts unseen data.  Your predictive model should have an MSE that is equal to or lower than 1400.

In [ ]:
import pandas as pd  
import seaborn as sns 
from sklearn.datasets import load_boston

In [ ]:
boston_dataset = load_boston()

In [ ]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

## Exercise 3:  Exploratory Data Analysis  (3 points)

### Summarize your Observations

* 1. 
* 2.
* 3.

# Prepare the Data for Subsequent Problems

We consider the following subset (for ethical reasons, we do not consider the variable B) that we split into a test, train and validation set.

In [ ]:
var_names = ["CRIM", "INDUS", "NOX", "RM", "AGE", "DIS", "TAX", "PTRATIO", "LSTAT"]
x = boston[var_names]
y = boston_dataset.target

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1) 

In [ ]:
x_train.shape

## Exercise 4: Use the Ridge Path to determine 3 variables that are most expressive (2 points)

In [ ]:
# your code

In [ ]:
# your code

plt.figure(figsize=(20,8))
plt.plot(np.arange(0.1, 25, 0.1), np.asarray(beta), 'o--')
plt.ylabel('beta')
plt.xlabel('ridge')
plt.axhline(y=0, c='black', linewidth=3, linestyle='dashed')
plt.legend(var_names, loc=1, fontsize=22)

Write down the variable names that you suggest to use:
* var 1
* var 2
* var 3

## Exercise 5:  Fit a simple model with 3 variables (3 points)

Build a simple model that uses the 3 most relevant variables and interpret the estimated coefficients. Use the bootstrap method to test whether the estimated parameters are significant different from 0.  Your model should have an MSE that is equal to or lower than 4100.

In [ ]:
subset_id = [0, 1, 2] # you need to chose the right indicies depending on your analyis above ....
x_train_sub = x_train[:,subset_id]
x_test_sub = x_test[:,subset_id]
x_val_sub = x_val[:,subset_id]

x_train_sub.shape

In [ ]:
skreg = Ridge(alpha=0.0)
skreg = skreg.fit(x_train_sub, y_train)
skreg.coef_

Compute the MSE on the test set:

In [ ]:
print('MSE', np.sum((y_test-skreg.predict(x_test_sub))**2))

Now, use the bootstrap method to check whether all coefficient are statistical significant. We say a coefficient is significant if the 95% bootstrap confidence interval does not contain zero. We indicate the lower 0.025 and upper 0.975 quantiles by red bars below.  

* https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/
* https://machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/

In [ ]:
skreg = Ridge(alpha=0.0)
beta = []
for i in range(2500):
    idx = np.random.choice(len(x_train_sub), len(x_train_sub), replace=True)
    beta.append(np.asarray(skreg.fit(x_train_sub[idx], y_train[idx]).coef_))

In [ ]:
beta = np.asarray(beta)
beta.shape

In [ ]:
plt.figure(figsize=(12,8))
sns.histplot(data=beta[:,0])
plt.axvline(x=np.quantile(beta[:,0],0.025), c='red')
plt.axvline(x=np.quantile(beta[:,0],0.975), c='red')

In [ ]:
# code for variable 2

In [ ]:
# code vor variable 3

Discuss your results:
* Are the signs of the coefficients practically significant?
* Are the coefficients statistical significant?
* Does a model with only 2 variables do better?

This is a bonus material, but you can also look to the partial residual plots. (https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html) You will see that the model is slightly misspecified, but real data are never going to be perfect.

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

regr = OLS(y_train, add_constant(x_train_sub)).fit()
print(regr.summary())

In [ ]:
import statsmodels.api as sm
fig = plt.figure(figsize=(18,7))
fig = sm.graphics.plot_partregress_grid(regr)
fig.tight_layout(pad=1.0)

## Exercise 6:  Build a predictive model (4 points)

The aim it to build a model that generalizes well to new data. You can use all variables or a subset of the variables and should also do some feature preprocessing. For instance, try 

* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

To get full points, your model should achieve an MSE below 1400 on the test set! But, it is not too difficult to get a model that achieves an MSE below 1200.

### Model 1

In [ ]:
subset_id = # your code
x_train_sub = x_train[:,subset_id]
x_test_sub = x_test[:,subset_id]
x_val_sub = x_val[:,subset_id]

In [ ]:
# your code, maybe do some data transformations here?

In [ ]:
# do some tuning on the validation set?

In [ ]:
pred_model = Ridge(alpha=0.0)
pred_model.fit(# your code)

In [ ]:
print('MSE', np.sum((y_test-pred_model.predict( your_array ))**2)) #replace your_array